# KNN - Memory(Item) Based Collaborative Filtering

In [1]:
import pandas as pd
import numpy as np
import math
from math import sqrt
from sklearn.model_selection import train_test_split as tts
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.neighbors import NearestNeighbors

In [2]:
import numpy as np

In [3]:
sampled_ratings=pd.read_csv("data/final_sr_red.csv") #reading csv sample rating
#sampled_ratings.drop("bin",axis=1, inplace=True)
sr=sampled_ratings

## Challenges with KNN:

## We face a lot of challenges with the knn model.

## 1. Runtime:  We use a KNN brute for algorithm to fill out martrix. We have 1000 X 137232 ratings to fill. Computing predictions for so many item-user pairs takes too long. This is a limitation of our model. To overcome this limitation, we evaluate our model/ tune parameters as discussed.


### We split of sampled data into a 80-20 train-validation split. To do this, we first create a pivot matrix of the enire sample data. We then unstack the pivoted data back to the original structure and randomly select 20% of the ratings as part of our validation set. We then remove these ratings in our pivot, fit our model on the train dataset, and try to predict these ratings in order to validate our model and tune our hyper-parameter 'k'.

## 2. Sparse matrix: Finding user similarity was difficult due to lack of values

In [4]:
#Pivoting sampled rating for KNN
sr = sr.pivot(
    index='movieId',
    columns='userId',
    values='rating'
).fillna(0)

In [5]:
# Unstacking the pivot
sr_test_list = sr.unstack().reset_index(name="rating").set_index("movieId")
sr_test_list = sr_test_list[sr_test_list.rating>0]

In [6]:
sr_test_list.shape

(916037, 2)

In [7]:
#splitting unstacked df into train-validation set
x, sr_test_rem = tts(sr_test_list, test_size=0.2, random_state = 69)

In [8]:
len(sr_test_rem)

183208

In [9]:
# setting our validation set ratings to 0(substitute for NA) 
a = list(sr_test_rem.index)
b = list(sr_test_rem.loc[:, "userId"])

for i in range(len(a)):
    sr.at[a[i],b[i]]=0

In [10]:
def goodness_of_fit(predictions,ratings):
    avg = np.mean(ratings)
    ss_res = 0
    for i in range(len(predictions)):
        err = (ratings[i]-predictions[i])**2
        ss_res += err
    ss_tot = 0
    for i in range(len(ratings)):
        ss_tot += (ratings[i]-avg)**2
    r2 = 1 - (ss_res / ss_tot)
    return r2

In [11]:
def fit_model(k):
    model_knn = NearestNeighbors(metric='cosine',algorithm='brute', n_neighbors=k, n_jobs=-1)
    model_knn.fit(sr)
    return model_knn
    
def recommendation(model, mid, k):
    if int(sr[sr.index == mid].sum(axis=1)) == 0:
        results = pd.DataFrame(list(zip([0]*k,[0]*k)), columns = ['sim', 'mov_id'])
    else:
        query_index=mid
        distances,indices=model.kneighbors(sr[sr.index==query_index].values.reshape(1,-1))
        ind = list(indices.flatten())
        distances = list(distances.flatten())
        sim=[]
        for i in range(len(distances)):
            sim.append(1-float(distances[i]))
        mov_ids = []
        for i in ind:
            mov_ids.append(sr.index[i])
        results = pd.DataFrame(list(zip(sim,mov_ids)), columns = ['sim', 'mov_id'])
        results = results.set_index("mov_id")
    return results

In [12]:
def cal_mean(s):
    count = 0
    add = 0 
    for i in range(len(s)):
        if s[i] != 0:
            add += s[i]
            count += 1
    return float(add)/(count+0.001)

def predict(mid, uid, r):
    w_sum = 0 
    sim_list = []
    if max(list(r.loc[:, "sim"]))==0:
        return cal_mean(list(sr[uid]))
    else:
        for m in list(r.index):
            if float(sr[sr.index == m][uid]) != 0.0:
                sim_list.append(float(r[r.index == m]['sim']))
                w_sum += float(sr[sr.index == m][uid])*float(r[r.index == m]['sim'])
        if sim_list == []:
            return cal_mean(list(sr[uid]))
        else:
            return w_sum/sum(sim_list)    

In [13]:
def multi_k(k):
    model = fit_model(k)
    act = []
    pred = []
    i=0
    total = len(sr_test_rem)
    m_list = list(set(sr_test_rem.index))

    for m in m_list:
        u_list = list(sr_test_rem[sr_test_rem.index==m]['userId'])
        print(i, "of", total ,"completed")
        i=i+len(u_list)
        r = recommendation(model, m, k)
        for u in u_list:
            p = predict(m, u, r)
            act.append(float(sr_test_rem[(sr_test_rem.index == m)&(sr_test_rem['userId'] ==u)]['rating']))
            pred.append(p)
            
    r2 = goodness_of_fit(pred, act)
    rmse = sqrt(mean_squared_error(act, pred))
    mae = mean_absolute_error(act, pred)
    print(k, " - RMSE:", rmse, " - MAE:", mae, " - R squared:", r2)
    return pred, act

In [14]:
p, a = multi_k(48) 

data = pd.DataFrame(list(zip(p, a)), columns=['pred', 'act'])
data.to_csv("recomm_results.csv", sep=',')

0 of 183208 completed
481 of 183208 completed
900 of 183208 completed
908 of 183208 completed
934 of 183208 completed
942 of 183208 completed
945 of 183208 completed
950 of 183208 completed
981 of 183208 completed
1914 of 183208 completed
2070 of 183208 completed
3135 of 183208 completed
3200 of 183208 completed
3237 of 183208 completed
3238 of 183208 completed
3241 of 183208 completed
3920 of 183208 completed
3931 of 183208 completed
4661 of 183208 completed
4877 of 183208 completed
4898 of 183208 completed
5176 of 183208 completed
5215 of 183208 completed
5737 of 183208 completed
6307 of 183208 completed
6477 of 183208 completed
6532 of 183208 completed
6534 of 183208 completed
6535 of 183208 completed
6546 of 183208 completed
7048 of 183208 completed
7880 of 183208 completed
7886 of 183208 completed
8008 of 183208 completed
8700 of 183208 completed
8732 of 183208 completed
8742 of 183208 completed
8851 of 183208 completed
9814 of 183208 completed
10053 of 183208 completed
10807 of 1

75258 of 183208 completed
75273 of 183208 completed
75675 of 183208 completed
75708 of 183208 completed
75729 of 183208 completed
75731 of 183208 completed
76958 of 183208 completed
76970 of 183208 completed
76992 of 183208 completed
76993 of 183208 completed
77240 of 183208 completed
77340 of 183208 completed
77743 of 183208 completed
77774 of 183208 completed
78345 of 183208 completed
78354 of 183208 completed
78359 of 183208 completed
78361 of 183208 completed
78362 of 183208 completed
78809 of 183208 completed
79820 of 183208 completed
79823 of 183208 completed
79825 of 183208 completed
80276 of 183208 completed
80285 of 183208 completed
80788 of 183208 completed
80795 of 183208 completed
80799 of 183208 completed
80800 of 183208 completed
81186 of 183208 completed
81847 of 183208 completed
81952 of 183208 completed
82478 of 183208 completed
82524 of 183208 completed
82538 of 183208 completed
82578 of 183208 completed
82598 of 183208 completed
82672 of 183208 completed
82681 of 183

146152 of 183208 completed
146219 of 183208 completed
146225 of 183208 completed
146268 of 183208 completed
146321 of 183208 completed
146326 of 183208 completed
146937 of 183208 completed
147109 of 183208 completed
148221 of 183208 completed
148272 of 183208 completed
148274 of 183208 completed
148278 of 183208 completed
148280 of 183208 completed
148284 of 183208 completed
148296 of 183208 completed
148297 of 183208 completed
148301 of 183208 completed
148304 of 183208 completed
148319 of 183208 completed
148320 of 183208 completed
148357 of 183208 completed
148361 of 183208 completed
148363 of 183208 completed
148745 of 183208 completed
149409 of 183208 completed
149478 of 183208 completed
149481 of 183208 completed
149515 of 183208 completed
149516 of 183208 completed
149597 of 183208 completed
149627 of 183208 completed
149631 of 183208 completed
149647 of 183208 completed
149648 of 183208 completed
149690 of 183208 completed
149983 of 183208 completed
149991 of 183208 completed
1

### After testing different k values: 8,16,32,48,64 we tune hyper-parameter 'k' to 48, considering our results for RMSE, R squared and MAE, plus a time trade-off (the amount of time our model takes to predict ratings, considering different values of k).

When k=48, we see that RMSE = 0.924, MAE = .704, R-squared = .196, time taken = 3 hours

When k=64, we see that RMSE = .922, MAE = .705, R-squared = .2, time taken = 5 hours

When k=32, we see that RMSE = .93, MAE = .707, R-squared = .183, time taken = 1.5 hours

When k=16, we see that RMSE = .963, MAE = .727, R-squared = .126, time taken = 40 mins

When k=08, we see that RMSE = 1.03, MAE = .77, R-squared = 0.003, time taken = 15-20 mins


In [21]:
data.head()

,pred,act
0,3.716396,2.5
1,3.161233,3.0
2,3.651837,3.0
3,3.886723,3.5
4,3.478329,3.0
